# DIMLP Fidex

In [1]:
import pandas as pd
import pathlib as pl

import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from dimlpfidex import fidex

# We'll try with this: https://hes-xplain.github.io/documentation/dimlpfidex/training-methods/randforeststrn/#arguments-list

from trainings import randForestsTrn # Part of DIMLPFIDEX

In [3]:
import pandas as pd
import pathlib as pl
from random import randint
from omnixai_community.data.tabular import Tabular
from omnixai_community.visualization.dashboard import Dashboard
from omnixai_community.explainers.tabular.auto import TabularExplainer
from mlxplain.explainers.tabular.specific.dimlpfidex import DimlpBTModel

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sonialikhan/heart-attack-analysis-and-prediction-dataset")

print("Path to dataset files:", path)

100%|██████████████████████████████████████████████████████████████| 4.11k/4.11k [00:00<00:00, 6.62MB/s]

Extracting files...
Path to dataset files: /home/twg/.cache/kagglehub/datasets/sonialikhan/heart-attack-analysis-and-prediction-dataset/versions/1
